In [1]:
%matplotlib inline
#LOAD LIBRARIES
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display
from sklearn import linear_model as sk
from sklearn.linear_model import SGDRegressor
from sklearn.utils import shuffle
import numpy as np

In [2]:
%%time
#CONNECT TO THE DATABASE
import sqlite3
disk_engine=sqlite3.connect('tuti.db')

Wall time: 1 ms


In [3]:
#CONNECTION TEST
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
df

,index,Year,Month,CRSDepTime,UniqueCarrier,Origin,Dest,Cancelled
0,1,1987,10,730,PS,SAN,SFO,0
1,2,1987,10,730,PS,SAN,SFO,0
2,3,1987,10,730,PS,SAN,SFO,0


In [9]:
%%time
model = sk.SGDClassifier()
classes= np.array([0,1])
for i in range(0,100,10):
    X_train = pd.read_sql_query('SELECT Year, Month, CRSDepTime '
                            'FROM data '
                           'LIMIT 10 OFFSET %i; ' %i, disk_engine)
    
    y_train = pd.read_sql_query('SELECT Cancelled '
                           'FROM data '
                           'LIMIT 10 OFFSET %i; ' %i, disk_engine)
    
    #X_train, y_train = shuffle(chunk_x, chunk_y)
    model.partial_fit(X_train, y_train.values.ravel(),classes=classes )


Wall time: 166 ms


In [10]:
print model.predict([[1987, 10, 1505]])

[0]


# NEW TEST

In [4]:
from sklearn.linear_model import SGDClassifier
from sklearn.utils.class_weight import compute_class_weight

from sklearn.utils.testing import assert_almost_equal
from sklearn.utils.testing import assert_true
from sklearn.utils.testing import assert_equal

In [46]:
def test_compute_class_weight():
    """Test (and demo) compute_class_weight."""
    classes, y = unique(np.asarray([0,1]), return_inverse=True)
    cw = compute_class_weight("auto", classes, y)
    assert_almost_equal(cw.sum(), classes.shape)
    assert_true(cw[0] < cw[1] < cw[2])

In [82]:
%%time

#create a list with the features we wan to use
feature_cols = ['Year','Month','CRSDepTime']

#define the classifier
classes = np.array([0,1])
weights = {0: 1, 1: 20}
SGDClass = SGDClassifier(class_weight=weights, 
                         #loss='log',
                         alpha=1.0
                         #l1_ratio=1,
                         #learning_rate='optimal',
                         #shuffle=True,
                         #eta0=0.01
                         )

#compute_class_weight('auto', classes, y)
for i in range(0,5000000,10000):
    train = pd.read_sql_query('SELECT * '
                              'FROM data '
                              'LIMIT 10000 OFFSET %i;  '%i, disk_engine)
    
    X = train.loc[:,feature_cols]
    y = train.Cancelled
    
    SGDClass.partial_fit(X,y,classes=classes)  

Wall time: 8min 1s


In [ ]:
%%time 

#load a test dataframe
test = pd.read_sql_query('SELECT * '
                        'FROM data '
                        'LIMIT 50000000 OFFSET 1000000 ', disk_engine)
X_new = test.loc[:,feature_cols]

In [85]:
#predict
new_pred_class = SGDClass.predict(X_new)
print new_pred_class

yes_count=0
for pred in new_pred_class:
    if pred==1:
        yes_count+=1
        
#see results
#print len(new_pred_class)
print yes_count
print 100*(len(test[test['Cancelled']==1]))/(float(len(test))),'% is the REAL percentage of cancellations'
print (yes_count*100)/(float(len(new_pred_class))),'% is the PREDICTED percentage of cancellations'

[0 0 0 ..., 0 0 0]
33630
1.02334 % is the REAL percentage of cancellations
0.6726 % is the PREDICTED percentage of cancellations


In [63]:
#example flight
print SGDClass.predict([[1987, 10, 1505]])

[0]
